<a href="https://colab.research.google.com/github/zahraDehghanian97/Poetry_Generator/blob/master/Char_Poem_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
embedding_dim = 256
rnn_units = 1024
seq_length = 1000

# make data ready

In [3]:
filepath = "/content/drive/MyDrive/Colab Notebooks/shahname.txt"
corpus = open(filepath, 'rb').read().decode(encoding='utf-8')
corpus = corpus.replace("\t"," \t ").replace("\n", " \n ")

In [4]:
vocab = sorted(set(corpus))
vocab_size = len(vocab)
print(len(vocab), "chars")
char2idx = {u:i for i, u in enumerate(vocab)}
print(char2idx)
idx2char = np.array(vocab)

48 chars
{'\t': 0, '\n': 1, ' ': 2, '(': 3, ')': 4, '«': 5, '»': 6, '،': 7, '؟': 8, 'ء': 9, 'آ': 10, 'أ': 11, 'ؤ': 12, 'ئ': 13, 'ا': 14, 'ب': 15, 'ت': 16, 'ث': 17, 'ج': 18, 'ح': 19, 'خ': 20, 'د': 21, 'ذ': 22, 'ر': 23, 'ز': 24, 'س': 25, 'ش': 26, 'ص': 27, 'ض': 28, 'ط': 29, 'ظ': 30, 'ع': 31, 'غ': 32, 'ف': 33, 'ق': 34, 'ل': 35, 'م': 36, 'ن': 37, 'ه': 38, 'و': 39, 'ٔ': 40, 'پ': 41, 'چ': 42, 'ژ': 43, 'ک': 44, 'گ': 45, 'ی': 46, '\u200c': 47}


In [5]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

text_as_int = np.array([char2idx[c] for c in corpus])
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

<BatchDataset shapes: (1001,), types: tf.int64>
<BatchDataset shapes: ((64, 1000), (64, 1000)), types: (tf.int64, tf.int64)>


# LSTM Model

In [6]:
def create_model_lstm(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE):
  model = keras.Sequential()
  model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size, None]))
  model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform',recurrent_regularizer=keras.regularizers.l2(0.01)))
  model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform',recurrent_regularizer=keras.regularizers.l2(0.01)))
  model.add(tf.keras.layers.Dense(vocab_size))
  return model

In [7]:
lstm_model = create_model_lstm()
lstm_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history = lstm_model.fit(dataset, epochs=50)
main_lstm_model = create_model_lstm(batch_size=1)
main_lstm_model.set_weights(lstm_model.get_weights())
# main_lstm_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/char_lstm.h5')
main_lstm_model.summary()

OSError: ignored

In [ ]:
def generate_text(model, start_string, temperature = 0.6):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(main_lstm_model, start_string=u"چنین گفت رستم به اسفندیار"))
main_lstm_model.save("/content/drive/MyDrive/Colab Notebooks/char_lstm.h5")